In [1]:
import os
from pyleaves.utils.csv_utils import (process_and_save_singledomain_datasets,
                                      process_and_save_multidomain_datasets,
                                      process_and_save_multidataset_singledomain_datasets,
                                      load_dataset)

from pyleaves.leavesdb import db_utils

OpenCV is built with OpenMP support. This usually results in poor performance. For details, see https://github.com/tensorpack/benchmarks/blob/master/ImageNet/benchmark-opencv-resize.py



In [3]:
local_db = db_utils.init_local_db()
output_dir = r'/media/data_cifs/jacob/Fossil_Project/replication_data'
# one_stage_val_splits = {'val_size':0.2, 'test_size':0.2}
# two_stage_val_splits = {'source':
#                                 {'val_size':0.2, 'test_size':0.0},
#                         'target':
#                                 {'val_size':0.2, 'test_size':0.2}
#                        }

one_stage_val_splits = {'val_size':0.0, 'test_size':0.5}
two_stage_val_splits = {'source':
                                {'val_size':0.0, 'test_size':0.5},
                        'target':
                                {'val_size':0.0, 'test_size':0.5}
                       }



dataset_names = ['PNAS','Leaves', 'Fossil']

dataset_name_pairs = [
                        ['PNAS','Leaves'],
                        ['PNAS','Fossil'],
                        ['Leaves','PNAS'],
                        ['Leaves','Fossil'],
                        ['Fossil','PNAS'],
                        ['Fossil','Leaves']
                     ]

dataset_name_merged_pairs = ['PNAS+Leaves','PNAS+Fossil','Leaves+Fossil']




## LOAD FULL DATASETS INTO DICT WITH {dataset_name:dataset_data} format
data_dict = {}
for name in dataset_names:
    data_dict[name] = load_dataset(local_db, name, x_col='path', y_col='family')

    
# process_and_save_multidataset_singledomain_datasets(data_dict=data_dict,
#                                                     dataset_names=dataset_names,
#                                                     validation_splits=one_stage_val_splits, 
#                                                     output_root_dir = os.path.join(output_dir,'2dataset-1domain_experiments'),
#                                                     merge_new_labels = True)

Copying sql db file from /home/jacob/pyleaves/pyleaves/leavesdb/resources/leavesdb.db to /home/jacob/scripts/leavesdb.db
Proceeding with sql db at location /home/jacob/scripts/leavesdb.db


In [5]:
process_and_save_singledomain_datasets(data_dict,
                                       dataset_names,
                                       validation_splits=one_stage_val_splits,
                                       output_root_dir=os.path.join(output_dir,'single-domain_experiments'))

## PARSE AND DISTRIBUTE DATA FILES FOR MULTIDOMAIN EXPERIMENTS
process_and_save_multidomain_datasets(data_dict,
                                     dataset_name_pairs,
                                     validation_splits=two_stage_val_splits,
                                     output_root_dir=os.path.join(output_dir,'2-domain_experiments'))

saved PNAS dataset train subset to /media/data_cifs/jacob/Fossil_Project/replication_data/single-domain_experiments/PNAS/PNAS/train_data.csv
saved PNAS dataset test subset to /media/data_cifs/jacob/Fossil_Project/replication_data/single-domain_experiments/PNAS/PNAS/test_data.csv
saved label maps to /media/data_cifs/jacob/Fossil_Project/replication_data/single-domain_experiments/PNAS/PNAS/label_mappings.csv
saved Leaves dataset train subset to /media/data_cifs/jacob/Fossil_Project/replication_data/single-domain_experiments/Leaves/Leaves/train_data.csv
saved Leaves dataset test subset to /media/data_cifs/jacob/Fossil_Project/replication_data/single-domain_experiments/Leaves/Leaves/test_data.csv
saved label maps to /media/data_cifs/jacob/Fossil_Project/replication_data/single-domain_experiments/Leaves/Leaves/label_mappings.csv
saved Fossil dataset train subset to /media/data_cifs/jacob/Fossil_Project/replication_data/single-domain_experiments/Fossil/Fossil/train_data.csv
saved Fossil data

In [ ]:
# def process_and_save_multidataset_singledomain_datasets(data_dict: dict, dataset_names: list, validation_splits: dict, output_root_dir: str, merge_new_labels: bool = True):
#     '''
#     Generate CSV datasets for single domain experiment, but for each pair of datasets to be merged
#     Arguments:
#         data_dict: dict
#         dataset_names: list
#             e.g. ['PNAS','Fossil','Leaves']. Pairs will be created within function.
#         validation_splits: dict
#         output_root_dir: str
#         merge_new_labels: bool = True
    
#     '''
#     for i, dataset_1 in enumerate(dataset_names):
#         for j, dataset_2 in enumerate(dataset_names):
#             if j==i:
#                 continue
#             dataset_name = '+'.join(dataset_1,dataset_2)
        
#             encoder = LabelEncoder()
#             experiment_dir = os.path.join(output_root_dir, dataset_name)
            
#             input_data = pd.concat([data_dict[dataset_1], data_dict[dataset_2]])
#             process_and_save_dataset(input_data,
#                                      name=dataset_name,
#                                      encoder=encoder,
#                                      validation_splits=validation_splits, 
#                                      experiment_dir=experiment_dir,
#                                      merge_new_labels=merge_new_labels,
#                                      other_data_keys=['dataset'])

In [5]:
process_and_save_multidataset_singledomain_datasets(data_dict=data_dict,
                                                    dataset_names=dataset_names,
                                                    validation_splits=one_stage_val_splits, 
                                                    output_root_dir = os.path.join(output_dir,'2dataset-1domain_experiments'),
                                                    merge_new_labels = True)

TypeError: join() takes exactly one argument (2 given)

In [5]:
import dataset
from stuf import stuf
import pandas as pd
from pyleaves.leavesdb import db_utils, db_query
from pyleaves import leavesdb

local_db = db_utils.init_local_db()

db = dataset.connect(f"sqlite:///{local_db}", row_type=stuf)
datasets = {}
for d in dataset_names:
    datasets.update({d:pd.DataFrame(db_query.load_data(db=db, x_col='path', y_col='family', dataset=d))})
data = pd.DataFrame(db['dataset'].all())
leavesdb.db_manager.analyze_db_contents(local_db)

# print(data.columns)

Copying sql db file from /home/jacob/pyleaves/pyleaves/leavesdb/resources/leavesdb.db to /home/jacob/scripts/leavesdb.db
Proceeding with sql db at location /home/jacob/scripts/leavesdb.db
FOUND:
44518 UNIQUE paths with 1 duplicates
----------
Keeping a total of 44518 paths and discarding 0 duplicates


In [19]:
from pyleaves.data_pipeline import preprocessing
import numpy as np
# for k,v in datasets.items():
#     print(k, v.shape)


filtered_data = preprocessing.filter_low_count_labels(datasets['Leaves'], threshold=20)

print(len(np.unique(filtered_data['family'])))

Selecting only samples that belong to a class with population >= 20 samples
Previous num_classes = 376, new num_classes = 144
Previous data_df.shape = (26953, 2), new data_df.shape = (25660, 2)
144


In [6]:
leavesdb.db_manager.analyze_db_contents(local_db)

FOUND:
44518 UNIQUE paths with 1 duplicates
----------
Keeping a total of 44518 paths and discarding 0 duplicates


In [7]:
db_utils.__get_num_families_per_dataset(db)

[('Fossil', 27), ('Leaves', 376), ('PNAS', 19), ('plant_village', 3)]

In [9]:
db_utils.summarize_db(db)

{'Database column keys': ['id', 'species', 'genus', 'path', 'family', 'dataset', 'label', 'source_path'], 'distinct datasets': ['Fossil', 'Leaves', 'PNAS', 'plant_village'], 'Number of distinct families': [('Fossil', 27), ('Leaves', 376), ('PNAS', 19), ('plant_village', 3)], 'Number of rows in db': 44518}


{'Database column keys': ['id',
  'species',
  'genus',
  'path',
  'family',
  'dataset',
  'label',
  'source_path'],
 'distinct datasets': ['Fossil', 'Leaves', 'PNAS', 'plant_village'],
 'Number of distinct families': [('Fossil', 27),
  ('Leaves', 376),
  ('PNAS', 19),
  ('plant_village', 3)],
 'Number of rows in db': 44518}

In [6]:
import dataset
import numpy as np
from stuf import stuf
import pandas as pd
from pyleaves.leavesdb import db_utils, db_query
from pyleaves import leavesdb

def load_data(db,
              datasets=['Fossil','Leaves'],
              x_col='path',
              y_col='family',
              keep_cols=['dataset']              
              ):
    data_df = pd.DataFrame(db['dataset'].all())
    data = []
    columns = [x_col, y_col, *keep_cols]
    #         data[data['dataset'] in datasets]
    for name in datasets:
        data += [
                    data_df[data_df.loc[:,'dataset'] == name]
                ]
    data = pd.concat(data)
    data = data.loc[:,columns]
    
    return data



x_col='path'
y_col='family'
keep_cols=['dataset']
datasets=['Fossil','Leaves']



local_db = db_utils.init_local_db()
db = dataset.connect(f"sqlite:///{local_db}", row_type=stuf)

data = load_data(db,
              datasets=['Fossil','Leaves', 'PNAS'],
              x_col='path',
              y_col='family',
              keep_cols=['dataset']              
              )

data.head()

Copying sql db file from /home/jacob/pyleaves/pyleaves/leavesdb/resources/leavesdb.db to /home/jacob/scripts/leavesdb.db
Proceeding with sql db at location /home/jacob/scripts/leavesdb.db


,path,family,dataset
5314,/media/data_cifs/jacob/Fossil_Project/opt_data...,Adoxaceae,Fossil
5315,/media/data_cifs/jacob/Fossil_Project/opt_data...,Adoxaceae,Fossil
5316,/media/data_cifs/jacob/Fossil_Project/opt_data...,Adoxaceae,Fossil
5317,/media/data_cifs/jacob/Fossil_Project/opt_data...,Adoxaceae,Fossil
5318,/media/data_cifs/jacob/Fossil_Project/opt_data...,Adoxaceae,Fossil


In [20]:
for d in data.groupby('dataset'):
    l, count = np.unique(d[1], return_counts=True)
    print(d[1].nunique())
    print(len(l[count>=100]))

path       6122
family       27
dataset       1
dtype: int64
11
path       26953
family       376
dataset        1
dtype: int64
62
path       5314
family       19
dataset       1
dtype: int64
20


In [8]:
np.unique(data['dataset'], return_counts=True)

(array(['Fossil', 'Leaves', 'PNAS'], dtype=object),
 array([ 6122, 26953,  5314]))